In [1]:
import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random

In [2]:
# Fix random seed
seed = 42
np.random.seed(seed)
random.seed(seed)
sampler = TPESampler(seed=seed)

# Pre-processing input data

In [3]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [4]:
results_df = pd.read_csv('../results_new/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio_new/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [5]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,45.91,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,25.77,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [6]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [7]:
mm_df = dataset_df[dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [8]:
st_df = dataset_df[~dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [9]:
# load test dataset
g_test = pd.read_csv('csv/g_test.csv')
st_test = pd.read_csv('csv/st_test.csv')
mm_test = pd.read_csv('csv/mm_test.csv')

In [10]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target).drop(['benchmark','brand_raw', 'brand_raw_target'])
features_st = features.copy().drop(['count', 'count_target'])

In [11]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler_g = StandardScaler()
X_g_train = scaler_g.fit_transform(X_g_train)
X_g_test = scaler_g.transform(X_g_test)

In [12]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler_st = StandardScaler()
X_st_train = scaler_st.fit_transform(X_st_train)
X_st_test = scaler_st.transform(X_st_test)

In [13]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler_mm = StandardScaler()
X_mm_train = scaler_mm.fit_transform(X_mm_train)
X_mm_test = scaler_mm.transform(X_mm_test)

# Model

In [14]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test):
	dtrain = xgb.DMatrix(X_train, label=y_train)
	dtest = xgb.DMatrix(X_test, label=y_test)
	
	# Definimos los hiperparámetros a buscar (XGBoost)
	param = {
		"verbosity": 0,
		"device": "gpu",
		"objective": "reg:squarederror",
		"eval_metric": "rmse",
		"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
		# L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
		"learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
	}
    
	if param["booster"] in ["gbtree", "dart"]:
		# maximum depth of the tree, signifies complexity of the tree.
		param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
		# minimum child weight, larger the term more conservative the tree.
		param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
		param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
		# defines how selective algorithm is.
		param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
		param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

	if param["booster"] == "dart":
		param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
		param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
		param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
		param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

	# training
	model = xgb.train(param, dtrain)
	# evaluation
	preds = model.predict(dtest)
	mse = mean_squared_error(y_test, preds)
	rmse = np.sqrt(mse)
	print(f"Trial: {trial.number} - MSE: {mse} - RMSE: {rmse}")
	return mse

# Hyperparameters Optimization

In [15]:
n_trials = 100
study_g = None
study_st = None
study_mm = None

## General

In [16]:
# configuration optuna
study_g = optuna.create_study(direction='minimize', sampler=sampler)
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test), n_trials=n_trials)

[I 2024-07-03 08:41:31,031] A new study created in memory with name: no-name-4cabfd89-10dc-4f5f-b8e6-9bead6aa32cb
[I 2024-07-03 08:41:31,344] Trial 0 finished with value: 88.35330896476874 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07, 'learning_rate': 0.002051110418843397, 'subsample': 0.24646688973455957, 'colsample_bytree': 0.8728673384861885}. Best is trial 0 with value: 88.35330896476874.


Trial: 0 - MSE: 88.35330896476874 - RMSE: 9.399644087132701


[I 2024-07-03 08:41:31,555] Trial 1 finished with value: 87.85497996247082 and parameters: {'booster': 'gblinear', 'lambda': 0.574485163632042, 'alpha': 0.04566054873446119, 'learning_rate': 0.0026587543983272706, 'subsample': 0.3454599737656805, 'colsample_bytree': 0.22423428436076215}. Best is trial 1 with value: 87.85497996247082.
[I 2024-07-03 08:41:31,686] Trial 2 finished with value: 88.15960708876744 and parameters: {'booster': 'gblinear', 'lambda': 2.1371407316372935e-06, 'alpha': 0.000784915956255507, 'learning_rate': 0.0019010245319870357, 'subsample': 0.43371571882817456, 'colsample_bytree': 0.3980437511290071}. Best is trial 1 with value: 87.85497996247082.
[I 2024-07-03 08:41:31,748] Trial 3 finished with value: 87.30562292052448 and parameters: {'booster': 'gblinear', 'lambda': 0.00012997969313168238, 'alpha': 0.0005486767416600901, 'learning_rate': 0.001238513729886093, 'subsample': 0.6860358815211507, 'colsample_bytree': 0.21199791750292696}. Best is trial 3 with value:

Trial: 1 - MSE: 87.85497996247082 - RMSE: 9.373098738542703
Trial: 2 - MSE: 88.15960708876744 - RMSE: 9.389334752194506
Trial: 3 - MSE: 87.30562292052448 - RMSE: 9.343747798422456


[I 2024-07-03 08:41:31,823] Trial 4 finished with value: 86.62905791789248 and parameters: {'booster': 'dart', 'lambda': 0.02932100047183291, 'alpha': 2.734702913886802e-06, 'learning_rate': 0.0015679933916723015, 'subsample': 0.7473864212097256, 'colsample_bytree': 0.4681448690526212, 'max_depth': 3, 'min_child_weight': 6, 'eta': 1.8841183049085085e-08, 'gamma': 0.1881755597772026, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.5710537951126793, 'skip_drop': 0.01588775693167255}. Best is trial 4 with value: 86.62905791789248.
[I 2024-07-03 08:41:31,957] Trial 5 finished with value: 87.36776608812744 and parameters: {'booster': 'gbtree', 'lambda': 0.23713402899336497, 'alpha': 5.1043449526824996e-08, 'learning_rate': 0.0024658447214487376, 'subsample': 0.23618183112843047, 'colsample_bytree': 0.3590638142251011, 'max_depth': 5, 'min_child_weight': 4, 'eta': 0.04264813784432918, 'gamma': 7.145401117237584e-06, 'grow_policy': 'lossguide'}.

...

Trial: 94 - MSE: 85.89757697738543 - RMSE: 9.26809457102081
Trial: 95 - MSE: 86.40284261845912 - RMSE: 9.29531293816723


[I 2024-07-03 08:41:40,205] Trial 96 finished with value: 87.31716464597388 and parameters: {'booster': 'gbtree', 'lambda': 1.5297688792877094e-05, 'alpha': 0.002901921844752947, 'learning_rate': 0.001594502610073617, 'subsample': 0.2895707817571205, 'colsample_bytree': 0.4286775231923314, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.028685260125854525, 'gamma': 0.00492828446998653, 'grow_policy': 'lossguide'}. Best is trial 76 with value: 85.82040514792926.
[I 2024-07-03 08:41:40,286] Trial 97 finished with value: 90.22374025520445 and parameters: {'booster': 'gbtree', 'lambda': 7.76519113720228e-05, 'alpha': 0.00038006165968127283, 'learning_rate': 0.02401364800110825, 'subsample': 0.25326978259019284, 'colsample_bytree': 0.1824106932951428, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.05501851424843677, 'gamma': 0.002060630390464048, 'grow_policy': 'lossguide'}. Best is trial 76 with value: 85.82040514792926.
[I 2024-07-03 08:41:40,384] Trial 98 finished with value: 86.0534572

Trial: 96 - MSE: 87.31716464597388 - RMSE: 9.344365395572558
Trial: 97 - MSE: 90.22374025520445 - RMSE: 9.49861780761835
Trial: 98 - MSE: 86.05345728909711 - RMSE: 9.27650027160551


[I 2024-07-03 08:41:40,523] Trial 99 finished with value: 85.86225255818445 and parameters: {'booster': 'gbtree', 'lambda': 7.311038960579585e-06, 'alpha': 0.003996618520918836, 'learning_rate': 0.001149897959782671, 'subsample': 0.356284072553164, 'colsample_bytree': 0.10790116031387059, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.017776398412860713, 'gamma': 0.00037059263556060105, 'grow_policy': 'lossguide'}. Best is trial 76 with value: 85.82040514792926.


Trial: 99 - MSE: 85.86225255818445 - RMSE: 9.266188674864356


In [17]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 76
Mejores parametros: {'booster': 'gbtree', 'lambda': 6.153362802392881e-07, 'alpha': 0.0011471508220748314, 'learning_rate': 0.001002737827637667, 'subsample': 0.20466176064508526, 'colsample_bytree': 0.11320379521367623, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.06983382228087344, 'gamma': 7.814049704902547e-05, 'grow_policy': 'lossguide'}
Mejor valor de pérdida: 85.82040514792926


## Single Thread

In [18]:
# configuration optuna
study_st = optuna.create_study(direction='minimize', sampler=sampler)
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test), n_trials=n_trials)

[I 2024-07-03 08:41:40,566] A new study created in memory with name: no-name-245d68df-af6a-41e7-b893-b81a2c8ae466
[I 2024-07-03 08:41:41,049] Trial 0 finished with value: 38.328243770616474 and parameters: {'booster': 'gblinear', 'lambda': 5.982221423148757e-07, 'alpha': 9.09751831698106e-08, 'learning_rate': 0.0047340227734041914, 'subsample': 0.9543277631300153, 'colsample_bytree': 0.35704278541971746}. Best is trial 0 with value: 38.328243770616474.
[I 2024-07-03 08:41:41,168] Trial 1 finished with value: 37.6305901348109 and parameters: {'booster': 'gblinear', 'lambda': 0.5946434805027183, 'alpha': 0.5007008855904077, 'learning_rate': 0.0031883397351001883, 'subsample': 0.5977988047139084, 'colsample_bytree': 0.33583439432593115}. Best is trial 1 with value: 37.6305901348109.
[I 2024-07-03 08:41:41,230] Trial 2 finished with value: 37.45153887219448 and parameters: {'booster': 'dart', 'lambda': 0.00010505873950469604, 'alpha': 2.5812496493273626e-08, 'learning_rate': 0.003608219327

Trial: 0 - MSE: 38.328243770616474 - RMSE: 6.190980840756695
Trial: 1 - MSE: 37.6305901348109 - RMSE: 6.134377730040017
Trial: 2 - MSE: 37.45153887219448 - RMSE: 6.119766243264074


[I 2024-07-03 08:41:41,298] Trial 3 finished with value: 41.784133482949656 and parameters: {'booster': 'gblinear', 'lambda': 3.105618343239014e-07, 'alpha': 2.1193424275165945e-08, 'learning_rate': 0.01519798062003422, 'subsample': 0.7420514894738259, 'colsample_bytree': 0.06575843748146334}. Best is trial 2 with value: 37.45153887219448.
[I 2024-07-03 08:41:41,356] Trial 4 finished with value: 37.50969335293937 and parameters: {'booster': 'dart', 'lambda': 2.482741090341392e-07, 'alpha': 0.0033690069338789532, 'learning_rate': 0.005935682912014687, 'subsample': 0.9493839909893877, 'colsample_bytree': 0.18064489693869357, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.24977498768365702, 'gamma': 0.10440344445480765, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 5.556766432156757e-08, 'skip_drop': 0.15056570556337193}. Best is trial 2 with value: 37.45153887219448.
[I 2024-07-03 08:41:41,458] Trial 5 finished with value: 56.27911635462043

Trial: 3 - MSE: 41.784133482949656 - RMSE: 6.464064780225339
Trial: 4 - MSE: 37.50969335293937 - RMSE: 6.124515764771887
Trial: 5 - MSE: 56.27911635462043 - RMSE: 7.501940839184246


[I 2024-07-03 08:41:41,500] Trial 6 finished with value: 45.685717689539906 and parameters: {'booster': 'gblinear', 'lambda': 1.9340353658219098e-07, 'alpha': 0.0002453976022270095, 'learning_rate': 0.02419860861453868, 'subsample': 0.7215690076020804, 'colsample_bytree': 0.2630558439875318}. Best is trial 2 with value: 37.45153887219448.
[I 2024-07-03 08:41:41,526] Trial 7 finished with value: 41.40192167675558 and parameters: {'booster': 'gbtree', 'lambda': 0.009374135805053809, 'alpha': 0.0015742119048074215, 'learning_rate': 0.04993980255757075, 'subsample': 0.7260903138402748, 'colsample_bytree': 0.589893173168698, 'max_depth': 3, 'min_child_weight': 5, 'eta': 1.323180041438319e-06, 'gamma': 8.951939677269576e-07, 'grow_policy': 'depthwise'}. Best is trial 2 with value: 37.45153887219448.
[I 2024-07-03 08:41:41,568] Trial 8 finished with value: 38.33426474339686 and parameters: {'booster': 'gbtree', 'lambda': 0.0001049776254156589, 'alpha': 0.00041231684401416913, 'learning_rate':

...

Trial: 92 - MSE: 37.31024625679498 - RMSE: 6.108211379511598
Trial: 93 - MSE: 37.30988479150545 - RMSE: 6.108181790967379
Trial: 94 - MSE: 37.30995333421158 - RMSE: 6.1081874016938595


[I 2024-07-03 08:41:51,128] Trial 95 finished with value: 37.30979422283714 and parameters: {'booster': 'gbtree', 'lambda': 1.782578542768778e-08, 'alpha': 0.0024165554049674254, 'learning_rate': 0.0011868123357540892, 'subsample': 0.27789307965424703, 'colsample_bytree': 0.050576426792806224, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.0022684573297189975, 'gamma': 3.635249160001904e-08, 'grow_policy': 'depthwise'}. Best is trial 95 with value: 37.30979422283714.
[I 2024-07-03 08:41:51,209] Trial 96 finished with value: 37.30948872114199 and parameters: {'booster': 'gbtree', 'lambda': 1.932116425524344e-08, 'alpha': 0.002607815095551034, 'learning_rate': 0.0010967660905229464, 'subsample': 0.27592137172367703, 'colsample_bytree': 0.07486753519541206, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.0027443109965453517, 'gamma': 3.330037397996609e-08, 'grow_policy': 'depthwise'}. Best is trial 96 with value: 37.30948872114199.
[I 2024-07-03 08:41:51,264] Trial 97 finished with value

Trial: 95 - MSE: 37.30979422283714 - RMSE: 6.108174377245392
Trial: 96 - MSE: 37.30948872114199 - RMSE: 6.108149369583392
Trial: 97 - MSE: 37.30990790165233 - RMSE: 6.108183682704076


[I 2024-07-03 08:41:51,338] Trial 98 finished with value: 37.31036714024318 and parameters: {'booster': 'gbtree', 'lambda': 1.9721446450373597e-08, 'alpha': 0.00260978115244722, 'learning_rate': 0.0010240341584971968, 'subsample': 0.23559221461102497, 'colsample_bytree': 0.05311324503234122, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.001316768502320952, 'gamma': 1.9571466746288824e-08, 'grow_policy': 'depthwise'}. Best is trial 96 with value: 37.30948872114199.
[I 2024-07-03 08:41:51,423] Trial 99 finished with value: 37.31097822822329 and parameters: {'booster': 'gbtree', 'lambda': 1.8757471518142993e-08, 'alpha': 0.002324272624225098, 'learning_rate': 0.0011210778573129939, 'subsample': 0.22970519104927184, 'colsample_bytree': 0.05009417306201422, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.001244574176201132, 'gamma': 1.60498210589029e-08, 'grow_policy': 'depthwise'}. Best is trial 96 with value: 37.30948872114199.


Trial: 98 - MSE: 37.31036714024318 - RMSE: 6.108221274662795
Trial: 99 - MSE: 37.31097822822329 - RMSE: 6.108271296219847


In [19]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 96
Mejores parametros: {'booster': 'gbtree', 'lambda': 1.932116425524344e-08, 'alpha': 0.002607815095551034, 'learning_rate': 0.0010967660905229464, 'subsample': 0.27592137172367703, 'colsample_bytree': 0.07486753519541206, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.0027443109965453517, 'gamma': 3.330037397996609e-08, 'grow_policy': 'depthwise'}
Mejor valor de pérdida en validación: 37.30948872114199


## Multi Thread

In [20]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize', sampler=sampler)
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test), n_trials=n_trials)

[I 2024-07-03 08:41:51,453] A new study created in memory with name: no-name-6d29bf4c-df2f-4ec8-baf7-964958a69904
[I 2024-07-03 08:41:51,573] Trial 0 finished with value: 677.1929151913695 and parameters: {'booster': 'gbtree', 'lambda': 5.989794878601609e-08, 'alpha': 0.0008318745034635068, 'learning_rate': 0.09552294429449876, 'subsample': 0.31206721218921923, 'colsample_bytree': 0.5424131697455499, 'max_depth': 9, 'min_child_weight': 8, 'eta': 0.0037681304379790182, 'gamma': 0.0041674718203826185, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 677.1929151913695.


Trial: 0 - MSE: 677.1929151913695 - RMSE: 26.022930565010725


[I 2024-07-03 08:41:51,702] Trial 1 finished with value: 565.3777931151645 and parameters: {'booster': 'dart', 'lambda': 0.202266714663928, 'alpha': 0.00012323645327131397, 'learning_rate': 0.010070072316456572, 'subsample': 0.8386361431734202, 'colsample_bytree': 0.6674657342388769, 'max_depth': 7, 'min_child_weight': 9, 'eta': 0.1318386461290972, 'gamma': 5.057795373282649e-06, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.960333750158439e-06, 'skip_drop': 0.0005329249885649307}. Best is trial 1 with value: 565.3777931151645.


Trial: 1 - MSE: 565.3777931151645 - RMSE: 23.77767425790766


[I 2024-07-03 08:41:51,887] Trial 2 finished with value: 568.724318139775 and parameters: {'booster': 'dart', 'lambda': 7.6124618871412325e-06, 'alpha': 1.0386855772080306e-07, 'learning_rate': 0.011078641782934488, 'subsample': 0.8159948424788888, 'colsample_bytree': 0.255029976122001, 'max_depth': 7, 'min_child_weight': 2, 'eta': 2.5909186033680598e-08, 'gamma': 0.00017817101436983106, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.02298738439458882, 'skip_drop': 1.4677700110205883e-06}. Best is trial 1 with value: 565.3777931151645.


...

Trial: 90 - MSE: 480.946050455006 - RMSE: 21.93048222121452
Trial: 91 - MSE: 469.23272542454015 - RMSE: 21.661780292130658
Trial: 92 - MSE: 498.8962697627925 - RMSE: 22.33598598143347
Trial: 93 - MSE: 487.41245301249785 - RMSE: 22.077419527936183
Trial: 94 - MSE: 519.0981198274637 - RMSE: 22.78372488921563
Trial: 95 - MSE: 492.3551378763966 - RMSE: 22.189076994692606


[I 2024-07-03 08:41:56,823] Trial 96 finished with value: 589.232300896488 and parameters: {'booster': 'dart', 'lambda': 0.0001372761118827814, 'alpha': 0.9607562069916135, 'learning_rate': 0.04912988436010749, 'subsample': 0.3089562442630713, 'colsample_bytree': 0.16420320935017216, 'max_depth': 7, 'min_child_weight': 3, 'eta': 0.0007394768071969343, 'gamma': 1.030008561992708e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.05980353113111701, 'skip_drop': 0.005647434803978711}. Best is trial 13 with value: 466.18796034552435.


Trial: 96 - MSE: 589.232300896488 - RMSE: 24.27410762307212


[I 2024-07-03 08:41:57,064] Trial 97 finished with value: 499.8274510909321 and parameters: {'booster': 'gblinear', 'lambda': 6.66630786815544e-05, 'alpha': 0.426081453294772, 'learning_rate': 0.06872804689278803, 'subsample': 0.24005181983140095, 'colsample_bytree': 0.10787238028232272}. Best is trial 13 with value: 466.18796034552435.
[I 2024-07-03 08:41:57,113] Trial 98 finished with value: 496.2742161628181 and parameters: {'booster': 'gblinear', 'lambda': 2.638559665077335e-06, 'alpha': 0.20784306805561004, 'learning_rate': 0.04473434326269602, 'subsample': 0.2000330336615164, 'colsample_bytree': 0.5427536638892467}. Best is trial 13 with value: 466.18796034552435.
[I 2024-07-03 08:41:57,149] Trial 99 finished with value: 486.49818252940344 and parameters: {'booster': 'gblinear', 'lambda': 1.422463407575128e-06, 'alpha': 0.7029382128490175, 'learning_rate': 0.07858966092962412, 'subsample': 0.28454495258062934, 'colsample_bytree': 0.06956832030377205}. Best is trial 13 with value:

Trial: 97 - MSE: 499.8274510909321 - RMSE: 22.356821131165585
Trial: 98 - MSE: 496.2742161628181 - RMSE: 22.277212935257815
Trial: 99 - MSE: 486.49818252940344 - RMSE: 22.056703800191983


In [21]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 13
Mejores parametros: {'booster': 'gblinear', 'lambda': 0.026758763366929273, 'alpha': 0.985176822519382, 'learning_rate': 0.05340673377810871, 'subsample': 0.21906996068917262, 'colsample_bytree': 0.15676169098118364}
Mejor valor de pérdida en validación: 466.18796034552435


# Training

In [22]:
models_folder = '../models/xgboost'

In [23]:
#dump(scaler_g, f'{models_folder}/scaler_g.joblib')
#dump(scaler_st, f'{models_folder}/scaler_st.joblib')
#dump(scaler_mm, f'{models_folder}/scaler_mm.joblib')

## General

In [24]:
# hyperparameters
if study_g is not None:
	params = study_g.best_trial.params
else:
	params = {
		'booster': 'gbtree',
		'lambda': 6.153362802392881e-07,
		'alpha': 0.0011471508220748314,
		'learning_rate': 0.001002737827637667,
		'subsample': 0.20466176064508526,
		'colsample_bytree': 0.11320379521367623,
		'max_depth': 9,
		'min_child_weight': 2,
		'eta': 0.06983382228087344,
		'gamma': 7.814049704902547e-05,
		'grow_policy': 'lossguide'
	}

In [25]:
# general models
dtrain = xgb.DMatrix(X_g_train, label=y_g_train)
dtest = xgb.DMatrix(X_g_test, label=y_g_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_g_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_g_test, preds)}")

MSE: 85.86453269271863 - RMSE: 9.2663117092357 - MAE: 6.330787289513482


In [26]:
# save model
model.save_model(f'{models_folder}/general.json')

## Single Thread

In [27]:
# hyperparameters
if study_st is not None:
	params = study_st.best_trial.params
else:
	params = {
		'booster': 'gbtree',
		'lambda': 1.932116425524344e-08,
		'alpha': 0.002607815095551034,
		'learning_rate': 0.0010967660905229464,
		'subsample': 0.27592137172367703,
		'colsample_bytree': 0.07486753519541206,
		'max_depth': 5,
		'min_child_weight': 9,
		'eta': 0.0027443109965453517,
		'gamma': 3.330037397996609e-08,
		'grow_policy': 'depthwise'
	}

In [28]:
# single thread model
dtrain = xgb.DMatrix(X_st_train, label=y_st_train)
dtest = xgb.DMatrix(X_st_test, label=y_st_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_st_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_st_test, preds)}")

MSE: 37.31206358760169 - RMSE: 6.108360138989981 - MAE: 3.2314192448368777


In [29]:
# save model
model.save_model(f'{models_folder}/single_thread.json')

## Multi Thread

In [30]:
# hyperparameters
if study_mm is not None:
	params = study_mm.best_trial.params
else:
	params = {
		'booster': 'gblinear',
		'lambda': 0.026758763366929273,
		'alpha': 0.985176822519382,
		'learning_rate': 0.05340673377810871,
		'subsample': 0.21906996068917262,
		'colsample_bytree': 0.15676169098118364
	}

In [31]:
# multi thread model
dtrain = xgb.DMatrix(X_mm_train, label=y_mm_train)
dtest = xgb.DMatrix(X_mm_test, label=y_mm_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_mm_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_mm_test, preds)}")

MSE: 466.6217449949687 - RMSE: 21.601429235005927 - MAE: 21.583701605902775


In [32]:
# save model
model.save_model(f'{models_folder}/multi_thread.json')

# Load models

In [34]:
model_g = xgb.Booster()
model_st = xgb.Booster()
model_mm = xgb.Booster()
model_g.load_model(f'{models_folder}/general.json')
model_st.load_model(f'{models_folder}/single_thread.json')
model_mm.load_model(f'{models_folder}/multi_thread.json')

In [35]:
def describe_val(model, X, y):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}
	
	dtest = xgb.DMatrix(X)
	predictions = model.predict(dtest)
	index_min = np.argmin(np.abs(predictions - y))
	min_instance["prediction"] = predictions[index_min]
	min_instance["actual"] = y[index_min]
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y))
	max_instance["prediction"] = predictions[index_max]
	max_instance["actual"] = y[index_max]
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [36]:
# general model
print("Validation set general model")
min_instance, max_instance, predictions = describe_val(model_g, X_g_test, y_g_test)
errors = np.abs(predictions - y_g_test)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_g_test)} | Std actual: {np.std(y_g_test)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("---")
print("Min instance")
print(g_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(g_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set general model
Mean prediction: 21.99935531616211 | Std actual: 0.017365766689181328
Mean actual: 27.556500000000003 | Std actual: 7.4113266524961645
Mean Error: 6.330787289513482 | Std Error: 6.766510547368797
---
Min instance
total_time                                                       15.74
total_cpu_usage                                                    1.0
max_ram_usage                                                31.339844
brand_raw                         12th Gen Intel(R) Core(TM) i5-12400F
count                                                               12
l2_cache_size                                                      7.5
l3_cache_size                                                     18.0
l2_cache_line_size                                                1280
l2_cache_associativity                                               7
benchmark                                                          TSP
ghz_actual_friendly                             

In [37]:
# single thread model
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val(model_st, X_st_test, y_st_test)
errors = np.abs(predictions - y_st_test)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_st_test)} | Std actual: {np.std(y_st_test)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(st_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(st_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model
Mean prediction: 24.191211700439453 | Std actual: 0.015047844499349594
Mean actual: 24.564000000000007 | Std actual: 6.096883138128859
Mean Error: 3.2314192448368777 | Std Error: 5.18362742215329
Min instance
total_time                                                        18.1
total_cpu_usage                                                    1.0
max_ram_usage                                                15.644531
brand_raw                         12th Gen Intel(R) Core(TM) i5-12400F
count                                                               12
l2_cache_size                                                      7.5
l3_cache_size                                                     18.0
l2_cache_line_size                                                1280
l2_cache_associativity                                               7
benchmark                                                     N_Queens
ghz_actual_friendly                           

In [38]:
# multi thread model
print("Validation set multi thread model")
min_instance, max_instance, predictions = describe_val(model_mm, X_mm_test, y_mm_test)
errors = np.abs(predictions - y_mm_test)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_mm_test)} | Std actual: {np.std(y_mm_test)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(mm_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(mm_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set multi thread model
Mean prediction: 14.950297355651855 | Std actual: 0.17648224532604218
Mean actual: 36.534 | Std actual: 0.8569854141115829
Mean Error: 21.583701605902775 | Std Error: 0.874968560760035
Min instance
total_time                                                        7.45
total_cpu_usage                                                  10.63
max_ram_usage                                              2385.507812
brand_raw                         12th Gen Intel(R) Core(TM) i5-12400F
count                                                               12
l2_cache_size                                                      7.5
l3_cache_size                                                     18.0
l2_cache_line_size                                                1280
l2_cache_associativity                                               7
benchmark                                                  MATRIX_MULT
ghz_actual_friendly                                       